In [87]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [153]:
# title_basics = pd.read_csv("../datas/raw/title.basics.tsv", sep='\t', low_memory=False)
# ###title_akas = pd.read_csv("../datas/raw/title.akas.tsv", sep='\t', low_memory=False)
# rating = pd.read_csv("../datas/raw/title.ratings.tsv", sep='\t')
# name = pd.read_csv("../datas/raw/name.basics.tsv", sep='\t')
# episodes = pd.read_csv("../datas/raw/title.episode.tsv", sep='\t')
# crew = pd.read_csv("../datas/raw/title.crew.tsv", sep='\t')
principal = pd.read_csv("../datas/raw/title.principals.tsv", sep='\t')

In [122]:
print(title_basics.info())
print(title_basics.head(10))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 628091 entries, 8 to 4883658
Data columns (total 7 columns):
tconst            628091 non-null object
titleType         628091 non-null object
primaryTitle      628091 non-null object
originalTitle     628034 non-null object
startYear         589060 non-null object
runtimeMinutes    628091 non-null uint16
genres            532854 non-null object
dtypes: object(6), uint16(1)
memory usage: 34.7+ MB
None
        tconst titleType                     primaryTitle  \
8    tt0000009     movie                       Miss Jerry   
332  tt0000335     movie            Soldiers of the Cross   
499  tt0000502     movie                         Bohemios   
571  tt0000574     movie      The Story of the Kelly Gang   
611  tt0000615     movie               Robbery Under Arms   
626  tt0000630     movie                           Hamlet   
669  tt0000675     movie                      Don Quijote   
670  tt0000676     movie  Don Álvaro o la fuerza del sino

In [155]:
# title_basics = title_basics.replace('\\N', np.NaN)
# title_basics = title_basics[title_basics["titleType"].isin(["movie", "video", "tvMovie", "tvSpecial"])]
# title_basics = title_basics[title_basics.isAdult == 0]
# title_basics.drop(["isAdult", "endYear"], axis=1, inplace=True)

# title_basics.runtimeMinutes = title_basics.runtimeMinutes.replace(np.NaN, 65535).astype(np.uint16)

# title_basics = title_basics.drop( title_basics[(title_basics["titleType"] == "video") & (title_basics["runtimeMinutes"] < 60)].index)
# title_basics = title_basics.drop( title_basics[(title_basics["titleType"] == "movie") & (title_basics["runtimeMinutes"] < 40)].index)
# title_basics = title_basics.drop( title_basics[(title_basics["titleType"] == "tvMovie") & (title_basics["runtimeMinutes"] < 60)].index)
# title_basics = title_basics.drop( title_basics[(title_basics["titleType"] == "tvSpecial") & (title_basics["runtimeMinutes"] < 60)].index)
# title_basics = title_basics.drop( title_basics[ (title_basics["runtimeMinutes"] > 65535 ) & (pd.isnull(title_basics["genres"])) ].index)

# title_basics.runtimeMinutes = title_basics.runtimeMinutes.replace(65535, np.NaN)

title_basics.to_csv("../datas/prepared/title.basics.csv", index=False)

In [14]:
print(title.info())
print(title.head(10))
print("\n\n")
print(rating.info())
print(rating.head())
print("\n\n")
print(name.info())
print(name.head())
print("\n\n")
print(episodes.info())
print(episodes.head())
print("\n\n")
print(crew.info())
print(crew.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3217093 entries, 0 to 3217092
Data columns (total 8 columns):
titleId            object
ordering           int64
title              object
region             object
language           object
types              object
attributes         object
isOriginalTitle    object
dtypes: int64(1), object(7)
memory usage: 196.4+ MB
None
     titleId  ordering                      title region language  \
0  tt0000001         1  Carmencita - spanyol tánc     HU       \N   
1  tt0000001         2                 Карменсита     RU       \N   
2  tt0000001         3                 Carmencita     US       \N   
3  tt0000001         4                 Carmencita     \N       \N   
4  tt0000002         1     Le clown et ses chiens     \N       \N   
5  tt0000002         2          A bohóc és kutyái     HU       \N   
6  tt0000002         3     Le clown et ses chiens     FR       \N   
7  tt0000002         4     The Clown and His Dogs     US       \N   
8  

In [142]:
# title_basics.set_index("tconst").join(rating.set_index('tconst'), on='tconst')

pd.merge(title_basics, rating, on='tconst', how='inner')

,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000009,movie,Miss Jerry,Miss Jerry,1894,45.0,Romance,5.4,62
1,tt0000335,movie,Soldiers of the Cross,Soldiers of the Cross,1900,NaN,"Biography,Drama",6.2,32
2,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,70.0,"Biography,Crime,Drama",6.3,439
3,tt0000615,movie,Robbery Under Arms,Robbery Under Arms,1907,NaN,Drama,5.1,12
4,tt0000630,movie,Hamlet,Amleto,1908,NaN,Drama,2.9,8
5,tt0000675,movie,Don Quijote,Don Quijote,1908,NaN,Drama,4.7,7
6,tt0000676,movie,Don Álvaro o la fuerza del sino,Don Álvaro o la fuerza del sino,1908,NaN,Drama,4.0,6
7,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,1908,120.0,"Adventure,Fantasy",6.1,24
8,tt0000739,movie,El pastorcito de Torrente,El pastorcito de Torrente,1908,NaN,Drama,7.2,6
9,tt0000793,movie,Andreas Hofer,Andreas Hofer,1909,NaN,Drama,4.8,6


In [143]:
title_basics

,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,1894,45.0,Romance
332,tt0000335,movie,Soldiers of the Cross,Soldiers of the Cross,1900,NaN,"Biography,Drama"
499,tt0000502,movie,Bohemios,Bohemios,1905,100.0,NaN
571,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,70.0,"Biography,Crime,Drama"
611,tt0000615,movie,Robbery Under Arms,Robbery Under Arms,1907,NaN,Drama
626,tt0000630,movie,Hamlet,Amleto,1908,NaN,Drama
669,tt0000675,movie,Don Quijote,Don Quijote,1908,NaN,Drama
670,tt0000676,movie,Don Álvaro o la fuerza del sino,Don Álvaro o la fuerza del sino,1908,NaN,Drama
673,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,1908,120.0,"Adventure,Fantasy"
730,tt0000739,movie,El pastorcito de Torrente,El pastorcito de Torrente,1908,NaN,Drama


In [148]:
name.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0043044,tt0050419,tt0053137,tt0072308"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0117057,tt0038355,tt0037382,tt0040506"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,producer","tt0057345,tt0049189,tt0063715,tt0059956"
3,nm0000004,John Belushi,1949,1982,"actor,writer,soundtrack","tt0077975,tt0078723,tt0080455,tt0072562"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0083922,tt0050976,tt0050986,tt0060827"


In [152]:
crew.head(10)

,tconst,directors,writers
0,tt0000001,nm0005690,\N
1,tt0000002,nm0721526,\N
2,tt0000003,nm0721526,\N
3,tt0000004,nm0721526,\N
4,tt0000005,nm0005690,\N
5,tt0000006,nm0005690,\N
6,tt0000007,"nm0005690,nm0374658",\N
7,tt0000008,nm0005690,\N
8,tt0000009,nm0085156,nm0085156
9,tt0000010,nm0525910,\N


In [154]:
principal.head()

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Herself""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N
